<a href="https://colab.research.google.com/github/newsun87/google-colab-line_bot_learning/blob/main/linebot_quick_reply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Quick Reply Message 設計

In [1]:
!pip install flask_ngrok pyngrok
!pip install Flask==2.0.3
!pip install line-bot-sdk
!ngrok authtoken 2Sh3hFOuE9JYeMNTsS6EzBxHPIL_6xdGTubFt7b9kSeUDFVYH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=860cb4c042bca93e98c3781f3774f487e836ca00ec6cfe3c153b57a0674174e4
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 2.2.5
    Uninstalling Flask-2.2.5:
      Successfully uninstalled Flask-2.2.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.6/730.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.0 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


#### Quick reply 最多可以支援 13 個 Action，並且支援 postback、message、datetimepicker、camera、camera rolla

Quick_Reply 搭配  MessageAction() 及 PostbackAction()

In [15]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

# 建立 Quick Reply 按鈕選項清單
item1 = QuickReplyButton(
        action=MessageAction(label="披薩", text="我想吃披薩"),
        image_url = 'https://i.imgur.com/02PGoH7.jpg'
    )
item2 =  QuickReplyButton(
        action=MessageAction(label="漢堡", text="我想吃漢堡"),
        image_url = 'https://i.imgur.com/02PGoH7.jpg'
    )
item3 =  QuickReplyButton(
        action=PostbackAction(label="炸雞", text="我想吃炸雞", data = '顯示炸雞的圖片'),
        image_url = 'https://i.imgur.com/02PGoH7.jpg'
    )
quick_reply_items = [item1, item2, item3] # 建立選項清單
# 建立 Quick Reply 介面
quick_reply = QuickReply(items=quick_reply_items)

# 監聽所有來自 LINE Bot Channel 的 Post Request
app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理文字訊息事件處理程序
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == '我想吃東西':
      # 建立快速選單回覆訊息
      quick_reply_message = TextSendMessage(text="請選擇你想吃的食物類型：", quick_reply=quick_reply)
      line_bot_api.reply_message(event.reply_token,  quick_reply_message)
    text_message = TextSendMessage(text="請好好享用")
    if event.message.text == '我想吃披薩':
       image_message = ImageSendMessage(
                        original_content_url = 'https://i.imgur.com/gISX3Rp.jpg',
                        preview_image_url = 'https://i.imgur.com/gISX3Rp.jpg')
       reply_message = [text_message, image_message]
       line_bot_api.reply_message(event.reply_token,  reply_message)
    if event.message.text == '我想吃漢堡':
       image_message = ImageSendMessage(
                        original_content_url = 'https://i.imgur.com/neejn90.jpg',
                        preview_image_url = 'https://i.imgur.com/neejn90.jpg')
       reply_message = [text_message, image_message]
       line_bot_api.reply_message(event.reply_token,  reply_message)

# 處理 Postback 事件
@line_handler.add(PostbackEvent)
def handle_postback_message(event):
    postback = event.postback.data # 取得 Postback 事件的資料
# -------------------------------------
    if postback == '顯示炸雞的圖片':
       image_message = ImageSendMessage(
                          original_content_url = 'https://i.imgur.com/tqMj24J.jpg',
                          preview_image_url = 'https://i.imgur.com/tqMj24J.jpg')
       text_message = TextSendMessage(text="請好好享用")
       reply_message = [text_message, image_message, ]
       line_bot_api.reply_message(event.reply_token, reply_message)

if __name__ == "__main__":
  app.run()

https://1c88-35-197-81-136.ngrok-free.app


<ipython-input-15-70f853e9c78e>:23: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-15-70f853e9c78e>:25: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-15-70f853e9c78e>:27: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_a

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470853457206050885', 'text': '我想吃東西'}, 'webhookEventId': '01H95P46D5BGEAV8CWGHA5HJ9K', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693481834674, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'bae50e138280463eb400bdbe4ae7de09', 'mode': 'active'}]}


...

<ipython-input-15-70f853e9c78e>:99: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, reply_message)
INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 11:37:42] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 11:37:42] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470853501984702804', 'text': '我想吃炸雞'}, 'webhookEventId': '01H95P50AF1TDM4J5WSZ71RQ28', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693481861402, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '512456f666354f0795509679ecc2d9a2', 'mode': 'active'}]}
訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470854075815034882', 'text': '我想吃東西'}, 'webhookEventId': '01H95PFEFNAH29SNG294RX5NVZ', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693482203492, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '4ae5af9063d94be399f6fe55f8e12d97', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 11:43:24] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470854958246264993', 'text': '我想吃東西'}, 'webhookEventId': '01H95PZG8B9CNXG86KZG16DMC4', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693482729281, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'a45384cb03ef4e9bb8ec3fca3527f28d', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [31/Aug/2023 11:52:10] "POST /callback HTTP/1.1" 200 -
